In [1]:
from model import Model

train_data_dir = "../data/train_data_chest_xray_set_60"
test_data_dir = "../data/chest_xray/test"
train_val_split_ratio = 0.8

model = Model(learning_rate=None, batch_size=32, patience_early_stopping=None,
              patience_reduce_learning_rate=None,
              factor_reduce_learning_rate=None, train_dir=train_data_dir,
              weight_decay=None, momentum=None, test_dir=test_data_dir,
              train_val_split_ratio=train_val_split_ratio)
model.load_existing_model("../models/model_set_60.pth")

TypeError: '<' not supported between instances of 'NoneType' and 'float'